In [94]:
import pandas as pd
import seaborn as sns
import numpy as np
from lib.preprocessing import *
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, SCORERS
from sklearn.model_selection import cross_validate, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

In [4]:
df = load_sample(percentage = 1, sql_db='data/cleaned_alex_emily_nelson.db')

In [82]:
feature_cols = ['order_no', 'user_id', 'Platform Type',
       'Placement - Day of Month', 'Placement - Weekday (Mo = 1)',
       'Placement - Time', 'Confirmation - Time', 'Arrival at Pickup - Time',
       'Pickup - Time', 'Distance (KM)', 'Temperature', 'rider_id',
       'Time from Pickup to Arrival', 'Fulfillment - Weekday (Su = 0)',
       'Fulfillment - Day of Month', 'orders', 'age', 'average_rating',
       'number_rating', 'pickup_distance_from_ref',
       'destination_distance_from_ref']
X = data[feature_cols]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)



In [83]:
X_scaled

array([[ 0.24927003, -1.60757572, -2.80275932, ..., -0.13420879,
        -0.45998297,  1.59359324],
       [-0.30100322,  1.25827409,  0.3964062 , ...,  0.47642783,
         0.23058182, -0.54003475],
       [ 1.31540914,  0.13002133,  0.3964062 , ...,  0.93564643,
        -0.02001773, -0.87950041],
       ...,
       [-0.56634415, -1.00613359, -2.80275932, ...,  0.63281038,
         2.37035778, -0.57607047],
       [-0.39785817,  0.82453334,  0.3964062 , ..., -0.74980995,
         0.26955155, -0.43569237],
       [ 1.28087931,  0.28455245,  0.3964062 , ...,  0.45408747,
        -0.02001773,  0.09330475]])

In [84]:
X_scaled = pd.DataFrame(X_scaled, columns=feature_cols)

In [85]:
X_scaled

,order_no,user_id,Platform Type,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Time,Arrival at Pickup - Time,Pickup - Time,Distance (KM),...,rider_id,Time from Pickup to Arrival,Fulfillment - Weekday (Su = 0),Fulfillment - Day of Month,orders,age,average_rating,number_rating,pickup_distance_from_ref,destination_distance_from_ref
0,0.249270,-1.607576,-2.802759,0.039359,2.399041,-1.031096,-1.029410,-1.067038,-1.008887,0.439856,...,-1.302480,0.314084,-2.043245,0.039343,-0.249908,0.001944,-0.854233,-0.134209,-0.459983,1.593593
1,-0.301003,1.258274,0.396406,0.948583,1.122927,-1.035167,-1.075038,-1.051293,-1.118946,-0.971370,...,1.078775,-1.006760,1.170605,0.948570,0.392929,-0.174352,0.237411,0.476428,0.230582,-0.540035
2,1.315409,0.130021,0.396406,0.039359,-0.153187,-0.351118,-0.386569,-0.463076,-0.473923,-0.265757,...,-0.056682,-0.245046,-0.114935,0.039343,1.035766,0.286493,-0.417575,0.935646,-0.020018,-0.879500
3,-0.803033,-1.006134,0.396406,-0.187948,0.484870,0.374515,0.336149,0.253172,0.311713,1.498275,...,-0.209737,-0.960165,0.527835,-0.187964,0.330042,0.714863,0.346576,0.498768,2.528508,1.572817
4,0.827461,-1.455679,-2.802759,0.607624,-1.429301,0.781335,0.740455,0.839641,0.768924,-0.794967,...,1.498787,-1.002708,-1.400475,0.607610,-1.022965,-0.161981,-0.963398,-0.787044,-0.020018,-0.477793
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21196,0.857093,-1.571577,0.396406,0.380318,1.760984,0.296737,0.313718,0.303138,0.248306,-0.618564,...,0.719273,-0.481056,1.813374,0.380303,3.198671,2.655672,0.019082,4.184928,-0.449108,0.466099
21197,1.592505,0.824533,0.396406,-1.665437,-1.429301,-0.075211,-0.092448,-0.134745,-0.119105,-0.265757,...,1.747946,-0.903442,-1.400475,-1.665459,0.136302,-0.703242,-0.090082,-0.089528,-0.385835,-0.644884
21198,-0.566344,-1.006134,-2.802759,0.266665,-0.153187,-1.185882,-1.033349,-1.000125,-0.985780,0.616259,...,0.434519,0.022364,-0.114935,0.266649,0.382765,0.280307,0.346576,0.632810,2.370358,-0.576070
21199,-0.397858,0.824533,0.396406,0.380318,-0.791244,-1.657059,-1.691290,-1.696365,-1.736262,-0.618564,...,0.751308,-0.016127,-0.757705,0.380303,-0.889570,-0.983152,1.219891,-0.749810,0.269552,-0.435692


In [10]:
df.head()

,order_no,user_id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Time,Arrival at Pickup - Time,...,Destination Lat,Destination Long,rider_id,Time from Pickup to Arrival,Fulfillment - Weekday (Su = 0),Fulfillment - Day of Month,orders,age,average_rating,number_rating
0,4423,3669,Bike,3,Business,16,3,10:13:30 AM,10:20:08 AM,10:27:13 AM,...,-1.270810,36.910537,379,2839,3,16,1601,1337,13.7,441
1,3319,1601,Bike,1,Personal,22,5,10:12:53 AM,10:13:11 AM,10:29:37 AM,...,-1.261375,36.795771,103,609,5,22,680,982,14.0,207
2,16705,1995,Bike,3,Business,25,4,11:56:31 AM,11:58:03 AM,11:59:17 AM,...,-1.279395,36.825364,709,1502,4,25,1023,294,14.2,145
3,24894,136,Bike,3,Business,10,4,1:46:27 PM,1:48:08 PM,1:48:28 PM,...,-1.325452,36.721658,829,813,4,10,1255,593,14.5,93
4,24641,3264,Bike,3,Business,12,5,2:48:05 PM,2:49:43 PM,3:17:52 PM,...,-1.308070,36.823719,99,1308,5,12,435,301,13.5,73


In [13]:
df.drop('Vehicle Type',axis=1, inplace=True)

In [17]:
#oreienting the pick and dest long and lat to get one data column 
reference_longitude = df['Pickup Long'].mean()
reference_latitude = df['Pickup Lat'].mean()
data['pickup_distance_from_ref'] = np.sqrt((df['Pickup Long'] - reference_longitude) ** 2 +
                                         (df['Pickup Lat'] - reference_latitude) ** 2)

In [21]:
reference_longitude1 = df['Destination Long'].mean()
reference_latitude1 = df['Destination Lat'].mean()
data['destination_distance_from_ref'] = np.sqrt((df['Destination Long'] - reference_longitude1) ** 2 +
                                         (df['Destination Lat'] - reference_latitude1) ** 2)

In [23]:
data.drop('Vehicle Type',axis=1, inplace=True)

In [56]:
data.columns

Index(['order_no', 'user_id', 'Platform Type', 'Personal or Business',
       'Placement - Day of Month', 'Placement - Weekday (Mo = 1)',
       'Placement - Time', 'Confirmation - Time', 'Arrival at Pickup - Time',
       'Pickup - Time', 'Distance (KM)', 'Temperature', 'rider_id',
       'Time from Pickup to Arrival', 'Fulfillment - Weekday (Su = 0)',
       'Fulfillment - Day of Month', 'orders', 'age', 'average_rating',
       'number_rating', 'pickup_distance_from_ref',
       'destination_distance_from_ref'],
      dtype='object')

In [31]:
#stealing alex's code, thanks bud
def get_seconds_from_dt_series(series: pd.Series) -> pd.Series:
        return pd.to_datetime(series).dt.hour * 3600 + pd.to_datetime(series).dt.minute * 60 + pd.to_datetime(series).dt.second
    
data["Placement - Time"]=get_seconds_from_dt_series(df["Placement - Time"])
data["Confirmation - Time"]=get_seconds_from_dt_series(df["Confirmation - Time"])
data["Arrival at Pickup - Time"]=get_seconds_from_dt_series(df["Arrival at Pickup - Time"])
data["Pickup - Time"]=get_seconds_from_dt_series(df["Pickup - Time"])

In [33]:
data.drop(['Pickup Lat', 'Pickup Long', 'Destination Lat', 'Destination Long'], axis=1, inplace=True)

In [38]:
# using this frame to create other frames to run the multi regression
data

,order_no,user_id,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Time,Arrival at Pickup - Time,Pickup - Time,...,rider_id,Time from Pickup to Arrival,Fulfillment - Weekday (Su = 0),Fulfillment - Day of Month,orders,age,average_rating,number_rating,pickup_distance_from_ref,destination_distance_from_ref
0,16147,19,1,Personal,16,7,36810,37208,37633,38924,...,101,1867,0,16,1299,986,13.1,287,0.027082,0.100012
1,11653,3283,3,Business,24,5,36773,36791,37777,37919,...,770,563,5,24,2311,872,14.1,533,0.046072,0.026236
2,24854,1998,3,Personal,16,3,42991,43083,43157,43809,...,451,1315,3,16,3323,1170,13.5,718,0.039181,0.014498
3,7553,704,3,Business,14,4,49587,49688,49708,50983,...,408,609,4,14,2212,1447,14.2,542,0.109262,0.099294
4,20869,192,1,Personal,21,1,53285,53383,55072,55158,...,888,567,1,21,82,880,13.0,24,0.039181,0.028388
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21196,21111,60,3,Business,19,6,48880,49483,50165,50404,...,669,1082,6,19,6728,2702,13.9,2027,0.027381,0.061026
21197,27117,2789,3,Business,1,1,45499,45771,46160,47049,...,958,665,1,1,1907,530,13.8,305,0.029121,0.022611
21198,9486,704,1,Business,18,3,35403,37172,38245,39135,...,589,1579,3,18,2295,1166,14.2,596,0.104913,0.024990
21199,10862,2789,3,Business,19,2,31120,31159,31877,32282,...,678,1541,2,19,292,349,15.0,39,0.047144,0.029844


In [106]:
def multiple_linear_regression(data, target_col, feature_cols, test_size=0.2, random_state=42):
   
    X = data[feature_cols]
    y = data[target_col]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    model = LinearRegression()
    model.fit(X_train, y_train)
    
    X_train = sm.add_constant(X_train)

    model = sm.OLS(y_train, X_train).fit()

    X_test = sm.add_constant(X_test)
    test_predictions = model.predict(X_test)
    

    test_mse = mean_squared_error(y_test, test_predictions)
    test_r2 = r2_score(y_test, test_predictions)
    rmse = mean_squared_error(y_test, test_predictions, squared=False)
    mae = mean_absolute_error(y_test, test_predictions)
    
    
  
    
    print("Mean Squared Error (MSE):", test_mse)
    print("Root Mean Squared Error (RMSE):", rmse)
    print("Mean Absolute Error (MAE):", mae)
    print("R-squared (R2):", test_r2)
    print(model.summary())




In [107]:
multiple_linear_regression(data, 'Time from Pickup to Arrival', ['Arrival at Pickup - Time', 'Distance (KM)', 'Temperature',
'pickup_distance_from_ref',
       'destination_distance_from_ref'])

Mean Squared Error (MSE): 622830.8162608468
Root Mean Squared Error (RMSE): 789.196310344167
Mean Absolute Error (MAE): 551.6202735346889
R-squared (R2): 0.3505212065638281
                                 OLS Regression Results                                
Dep. Variable:     Time from Pickup to Arrival   R-squared:                       0.334
Model:                                     OLS   Adj. R-squared:                  0.334
Method:                          Least Squares   F-statistic:                     1700.
Date:                         Thu, 20 Jul 2023   Prob (F-statistic):               0.00
Time:                                 20:22:30   Log-Likelihood:            -1.3759e+05
No. Observations:                        16960   AIC:                         2.752e+05
Df Residuals:                            16954   BIC:                         2.752e+05
Df Model:                                    5                                         
Covariance Type:                   

In [62]:
multiple_linear_regression(data, 'Time from Pickup to Arrival', ['Placement - Weekday (Mo = 1)','Distance (KM)','age', 'average_rating','pickup_distance_from_ref'])

Mean Squared Error (MSE): 618072.7451917119
Root Mean Squared Error (RMSE): 786.1760268487662
Mean Absolute Error (MAE): 549.0084642074303
R-squared (R2): 0.3554828529313242


(LinearRegression(),
 array([2011.97499104, 1303.76376357, 1754.43292781, ..., 2698.23742142,
        1517.43679538, 1151.48813234]),
 618072.7451917119,
 0.3554828529313242,
 array([ 2.24752132e+00,  1.01006850e+02, -4.48194343e-02, -3.48891517e+01,
         1.71410976e+02]),
 1111.1387409449953)

In [63]:
multiple_linear_regression(data, 'Time from Pickup to Arrival', [ 'Distance (KM)', 'Temperature',
'pickup_distance_from_ref',
       'destination_distance_from_ref'])

Mean Squared Error (MSE): 622846.4850407325
Root Mean Squared Error (RMSE): 789.206237330099
Mean Absolute Error (MAE): 551.6295424034097
R-squared (R2): 0.35050486739115205


(LinearRegression(),
 array([1992.89711464, 1305.39455032, 1707.68017925, ..., 2722.72867712,
        1502.72083473, 1201.04840417]),
 622846.4850407325,
 0.35050486739115205,
 array([100.87542363,   4.78207939, 157.69011123,  94.77051787]),
 475.9306054043943)

In [68]:
data.columns

Index(['order_no', 'user_id', 'Platform Type', 'Personal or Business',
       'Placement - Day of Month', 'Placement - Weekday (Mo = 1)',
       'Placement - Time', 'Confirmation - Time', 'Arrival at Pickup - Time',
       'Pickup - Time', 'Distance (KM)', 'Temperature', 'rider_id',
       'Time from Pickup to Arrival', 'Fulfillment - Weekday (Su = 0)',
       'Fulfillment - Day of Month', 'orders', 'age', 'average_rating',
       'number_rating', 'pickup_distance_from_ref',
       'destination_distance_from_ref'],
      dtype='object')

In [87]:
multiple_linear_regression(data, 'Time from Pickup to Arrival', ['order_no', 'user_id', 'Platform Type', 
       'Placement - Day of Month', 'Placement - Weekday (Mo = 1)',
       'Placement - Time', 'Confirmation - Time', 'Arrival at Pickup - Time',
       'Pickup - Time', 'Distance (KM)', 'Temperature', 'rider_id',
        'Fulfillment - Weekday (Su = 0)',
       'Fulfillment - Day of Month', 'orders', 'age', 'average_rating',
       'number_rating', 'pickup_distance_from_ref',
       'destination_distance_from_ref'])

Mean Squared Error (MSE): 612531.7069910184
Root Mean Squared Error (RMSE): 782.6440487162848
Mean Absolute Error (MAE): 548.1923217407667
R-squared (R2): 0.36126096588112244
Coefficients: [-1.45429000e-03  1.50267829e-02 -7.92535794e+00 -3.73825963e+02
 -8.69190588e+00  5.94761308e-03 -7.03983911e-03  1.56551536e-02
 -1.44853287e-02  1.01006780e+02  6.51911475e+00  2.91641281e-02
  1.13652473e+01  3.72687963e+02 -9.98954051e-02  1.41265383e-02
 -3.40057311e+01  2.58214844e-01  1.98833419e+02  8.27290428e+01]
Intercept: 992.4700439113471


In [71]:
multiple_linear_regression(data, 'Time from Pickup to Arrival', ['Platform Type', 
      'Placement - Weekday (Mo = 1)',
       'Placement - Time',
       'Pickup - Time', 'Distance (KM)', 'Temperature', 'rider_id', 'orders', 'age', 'average_rating',
       'number_rating', 'pickup_distance_from_ref',
       'destination_distance_from_ref'])

Mean Squared Error (MSE): 612686.1198993477
Root Mean Squared Error (RMSE): 782.7426907351787
Mean Absolute Error (MAE): 548.4423497362191
R-squared (R2): 0.36109994637666887
Coefficients: [-7.63472871e+00  1.54087421e+00  6.38815209e-03 -6.29289091e-03
  1.00908611e+02  6.54403949e+00  3.02531550e-02 -1.02052769e-01
  1.48887377e-02 -3.39710903e+01  2.66029667e-01  2.36060328e+02
  7.01576801e+01]
Intercept: 982.0995417529441


In [92]:
multiple_linear_regression(data, 'Time from Pickup to Arrival',['orders', 'age', 'average_rating',
       'number_rating'])

Mean Squared Error (MSE): 945447.3485345171
Root Mean Squared Error (RMSE): 972.3411687954579
Mean Absolute Error (MAE): 743.0892762879239
R-squared (R2): 0.014101442715933943
Coefficients: [-1.10051622e-01  1.78376255e-02 -4.27823090e+01  3.72197240e-01]
Intercept: 2194.355175812134


In [77]:
multiple_linear_regression(data, 'Time from Pickup to Arrival', ['Placement - Time','Distance (KM)','Temperature','average_rating','number_rating'])

Mean Squared Error (MSE): 619803.3612851343
Root Mean Squared Error (RMSE): 787.2759117902277
Mean Absolute Error (MAE): 549.4454224005809
R-squared (R2): 0.3536781919818145
Coefficients: [ 2.68016347e-06  1.01082965e+02  4.59354207e+00 -3.17594411e+01
 -6.03241654e-02]
Intercept: 950.5062480059119


In [78]:
multiple_linear_regression(data, 'Time from Pickup to Arrival', ['Placement - Day of Month', 'Placement - Weekday (Mo = 1)',
       'Placement - Time', 'Confirmation - Time', 'Arrival at Pickup - Time',
       'Pickup - Time',])

Mean Squared Error (MSE): 958935.9454571841
Root Mean Squared Error (RMSE): 979.252748506321
Mean Absolute Error (MAE): 745.8671467258182
R-squared (R2): 3.573269362888709e-05
Coefficients: [-1.90187927e+00  4.77157176e+00 -3.26785040e-03  2.66370723e-03
  1.51717504e-02 -1.45377496e-02]
Intercept: 1579.7820000993001


In [73]:
# 'order_no', 'user_id', 'Platform Type', 'Personal or Business',
#        'Placement - Day of Month', 'Placement - Weekday (Mo = 1)',
#        'Placement - Time', 'Confirmation - Time', 'Arrival at Pickup - Time',
#        'Pickup - Time', 'Distance (KM)', 'Temperature', 'rider_id',
#        'Time from Pickup to Arrival', 'Fulfillment - Weekday (Su = 0)',
#        'Fulfillment - Day of Month', 'orders', 'age', 'average_rating',
#        'number_rating', 'pickup_distance_from_ref',
#        'destination_distance_from_ref'],

# multiple_linear_regression(data, 'Time from Pickup to Arrival',

In [86]:
multiple_linear_regression(X_scaled, 'Time from Pickup to Arrival', ['rider_id', 'Placement - Weekday (Mo = 1)','pickup_distance_from_ref',])

Mean Squared Error (MSE): 0.9837504235513553
Root Mean Squared Error (RMSE): 0.9918419347614595
Mean Absolute Error (MAE): 0.7556507589793416
R-squared (R2): 0.0001567844719380096
Coefficients: [0.01996082 0.00707469 0.0058081 ]
Intercept: 0.0012200962858209374


In [89]:
sns.relplot(x=y_test, y=test_predictions, kind="scatter")

NameError: name 'y_test' is not defined